In [1]:
#importing the libraries needed for ML
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv(r'..\Datasets\Parkinsons Train Data.csv', header = 0)
data.head()

,Jitter(local),"Jitter(local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, db)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,status
0,1.488,0.000090,0.900,0.794,2.699,8.334,0.779,4.517,4.609,6.802,...,142.229,187.576,160,159,0.006065,0.000416,0.000,0,0.000,1
1,0.728,0.000038,0.353,0.376,1.059,5.864,0.642,2.058,3.180,7.194,...,159.515,234.505,170,169,0.005181,0.000403,2.247,0,0.000,1
2,1.220,0.000074,0.732,0.670,2.196,8.719,0.875,4.347,5.166,7.548,...,146.445,211.442,1431,1427,0.006071,0.000474,10.656,1,0.178,1
3,2.502,0.000123,1.156,1.634,3.469,13.513,1.273,5.263,8.771,16.779,...,182.713,220.230,94,92,0.004910,0.000320,0.000,0,0.000,1
4,3.509,0.000167,1.715,1.539,5.145,9.112,1.040,3.102,4.927,12.823,...,182.821,225.162,117,114,0.004757,0.000380,18.182,1,13.318,1


In [3]:
#setting the dependent variable
y = data.iloc[:, -1].values
y

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [4]:
X = data.iloc[:, :-1].values
X

array([[1.48800e+00, 9.02130e-05, 9.00000e-01, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [7.28000e-01, 3.76980e-05, 3.53000e-01, ..., 2.24700e+00,
        0.00000e+00, 0.00000e+00],
       [1.22000e+00, 7.40410e-05, 7.32000e-01, ..., 1.06560e+01,
        1.00000e+00, 1.78000e-01],
       ...,
       [2.91200e+00, 2.26205e-04, 1.14100e+00, ..., 2.00000e+01,
        1.00000e+00, 4.57400e+00],
       [3.78800e+00, 2.74883e-04, 1.78600e+00, ..., 1.66670e+01,
        3.00000e+00, 1.95750e+01],
       [1.80900e+00, 1.48510e-04, 6.80000e-01, ..., 1.81820e+01,
        1.00000e+00, 3.38700e+00]])

In [5]:
X.shape

(1036, 26)

In [6]:
from sklearn.preprocessing import StandardScaler
standard_X = StandardScaler()
standard_scaled_X = standard_X.fit_transform(X)
standard_scaled_X

array([[-0.67709317, -0.75525109, -0.35596897, ..., -1.32200138,
        -0.70349085, -0.81748742],
       [-1.10769097, -1.24846346, -0.91419898, ..., -1.21483914,
        -0.70349085, -0.81748742],
       [-0.82893555, -0.9071359 , -0.52741804, ..., -0.81380343,
        -0.0848011 , -0.80575498],
       ...,
       [ 0.12971113,  0.52196373, -0.11002119, ..., -0.3681764 ,
        -0.0848011 , -0.51600327],
       [ 0.62603176,  0.97913962,  0.54822078, ..., -0.52713133,
         1.15257839,  0.47275138],
       [-0.49522226, -0.20773512, -0.58048561, ..., -0.45487909,
        -0.0848011 , -0.5942415 ]])

In [7]:
def standardScale(X):
    X = standard_X.transform(X)
    return X

In [8]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
minmax_scaled_X = min_max_scaler.fit_transform(X)
minmax_scaled_X

array([[0.09149866, 0.108709  , 0.1053558 , ..., 0.        , 0.        ,
        0.        ],
       [0.03792471, 0.04051754, 0.03658537, ..., 0.02548833, 0.        ,
        0.        ],
       [0.0726068 , 0.08770943, 0.08423435, ..., 0.12087389, 0.08333333,
        0.00257534],
       ...,
       [0.19187932, 0.28529654, 0.13565502, ..., 0.2268654 , 0.08333333,
        0.06617764],
       [0.25363034, 0.34850561, 0.21674629, ..., 0.18905828, 0.25      ,
        0.28321542],
       [0.1141266 , 0.18440848, 0.07769676, ..., 0.20624334, 0.08333333,
        0.04900386]])

In [9]:
def minmaxScale(X):
    X = min_max_scaler.transform(X)
    return X

In [10]:
data.columns

Index(['Jitter(local)', 'Jitter(local, absolute)', 'Jitter (rap)',
       'Jitter (ppq5)', 'Jitter (ddp)', 'Shimmer (local)',
       'Shimmer (local, db)', 'Shimmer (apq3)', 'Shimmer (apq5)',
       'Shimmer (apq11)', 'Shimmer (dda)', 'AC', 'NDH', 'HTM', 'Median Pitch',
       'Mean Pitch', 'Standard deviation', 'Minimum pitch', 'Maximum pitch',
       'Number of pulses', 'Number of periods', 'Mean period',
       'Standard deviation of period', 'Fraction of locally unvoiced frames',
       'Number of voice breaks', 'Degree of voice breaks', 'status'],
      dtype='object')

In [11]:
fixed_cols = ['AC', 'NDH', 'HTM', 'Minimum pitch', 'Maximum pitch', 'Number of pulses', 'Number of periods',
              'Mean period', 'Fraction of locally unvoiced frames', 'Number of voice breaks', 'Degree of voice breaks']
jitter_cols = ['Jitter(local)', 'Jitter(local, absolute)', 'Jitter (rap)','Jitter (ppq5)', 'Jitter (ddp)']
shimmer_cols = ['Shimmer (local)', 'Shimmer (local, db)', 'Shimmer (apq3)', 'Shimmer (apq5)', 'Shimmer (apq11)',
                'Shimmer (dda)']
pitch_cols = ['Median Pitch', 'Mean Pitch']

In [12]:
def getBestNeighbors(X_train, X_test, y_train, y_test):
    from sklearn.neighbors import KNeighborsClassifier
    training_accuracy = []
    test_accuracy = []
    # try n_neighbors from 1 to 100
    neighbors_settings = range(1, 100)
    index = 0
    max_accuracy = 0
    
    for n_neighbors in neighbors_settings:
        # build the model
        knn = KNeighborsClassifier(n_neighbors=n_neighbors)
        knn.fit(X_train, y_train)
        # record training set accuracy
        training_accuracy.append(knn.score(X_train, y_train))
        # record test set accuracy
        accuracy = knn.score(X_test, y_test)
        test_accuracy.append(accuracy)
        if(accuracy > max_accuracy):
            max_accuracy = accuracy
            index = n_neighbors
        
    print("Best Accuracy:", max_accuracy)
    print("N_neighbors: ", index)
    return index

In [13]:
def getKNN(X, y):
    
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=12, test_size = 0.2)
    # Train
    from sklearn.neighbors import KNeighborsClassifier
    bestNoOfNeighbors = getBestNeighbors(X_train, X_test, Y_train, Y_test)
    model = KNeighborsClassifier(n_neighbors=bestNoOfNeighbors)
    model.fit(X_train, Y_train)

    # Evaluate
    print("Accuracy for training set -> ", end = "")
    Y_hat = [round(yhat) for yhat in model.predict(X)]
    print(accuracy_score(y, Y_hat)) # Full set accuracy
    Y_hat = [round(yhat) for yhat in model.predict(X_test)]
    print("Accuracy for test set -> ", end = "")
    print(accuracy_score(Y_test, Y_hat)) # Test set accuracy

    #Validations
    y_pred = model.predict(X_test)

    #Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(Y_test, y_pred) # Calulate Confusion matrix for test set.
    print(cm)

    from sklearn.model_selection import cross_val_score
    accuracies = cross_val_score(estimator = model, X = X_train, y = Y_train, cv = 10)
    print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
    print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

In [14]:
for jitter in jitter_cols:
    for shimmer in shimmer_cols:
        for pitch in pitch_cols:
            
            fixed_cols.append(jitter)
            fixed_cols.append(shimmer)
            fixed_cols.append(pitch)
            print(fixed_cols)
            print("\n")
            print("Added columns to the dataset are:", jitter, shimmer, pitch)
            print("\n")
            X1 = data[fixed_cols].values
            print("---------Standard Scaled---------")
            print("\n")
            standard_X = StandardScaler()
            standard_scale_X = standard_X.fit_transform(X1)
            standard_scale_X
            getKNN(standard_scale_X, y)
            print("\n")
            print("---------MinMax Scaled---------")
            print("\n")
            min_max_scaler = preprocessing.MinMaxScaler()
            minmax_scale_X = min_max_scaler.fit_transform(X1)
            minmax_scale_X
            getKNN(minmax_scale_X, y)
            fixed_cols.remove(jitter)
            fixed_cols.remove(shimmer)
            fixed_cols.remove(pitch)
            print("\n")

['AC', 'NDH', 'HTM', 'Minimum pitch', 'Maximum pitch', 'Number of pulses', 'Number of periods', 'Mean period', 'Fraction of locally unvoiced frames', 'Number of voice breaks', 'Degree of voice breaks', 'Jitter(local)', 'Shimmer (local)', 'Median Pitch']


Added columns to the dataset are: Jitter(local) Shimmer (local) Median Pitch


---------Standard Scaled---------


Best Accuracy: 0.7067307692307693
N_neighbors:  7
Accuracy for training set -> 0.7403474903474904
Accuracy for test set -> 0.7067307692307693
[[74 32]
 [29 73]]
Accuracy: 62.93 %
Standard Deviation: 2.54 %


---------MinMax Scaled---------


Best Accuracy: 0.7115384615384616
N_neighbors:  7
Accuracy for training set -> 0.7374517374517374
Accuracy for test set -> 0.7115384615384616
[[76 30]
 [30 72]]
Accuracy: 63.41 %
Standard Deviation: 4.49 %


['AC', 'NDH', 'HTM', 'Minimum pitch', 'Maximum pitch', 'Number of pulses', 'Number of periods', 'Mean period', 'Fraction of locally unvoiced frames', 'Number of voice breaks', 'De

Best Accuracy: 0.7211538461538461
N_neighbors:  17
Accuracy for training set -> 0.7162162162162162
Accuracy for test set -> 0.7211538461538461
[[74 32]
 [26 76]]
Accuracy: 63.05 %
Standard Deviation: 5.54 %


---------MinMax Scaled---------


Best Accuracy: 0.6971153846153846
N_neighbors:  11
Accuracy for training set -> 0.7191119691119691
Accuracy for test set -> 0.6971153846153846
[[76 30]
 [33 69]]
Accuracy: 62.57 %
Standard Deviation: 3.52 %


['AC', 'NDH', 'HTM', 'Minimum pitch', 'Maximum pitch', 'Number of pulses', 'Number of periods', 'Mean period', 'Fraction of locally unvoiced frames', 'Number of voice breaks', 'Degree of voice breaks', 'Jitter(local)', 'Shimmer (dda)', 'Mean Pitch']


Added columns to the dataset are: Jitter(local) Shimmer (dda) Mean Pitch


---------Standard Scaled---------


Best Accuracy: 0.7211538461538461
N_neighbors:  17
Accuracy for training set -> 0.7113899613899614
Accuracy for test set -> 0.7211538461538461
[[74 32]
 [26 76]]
Accuracy: 62.20 %
Stand

Best Accuracy: 0.6923076923076923
N_neighbors:  18
Accuracy for training set -> 0.694015444015444
Accuracy for test set -> 0.6923076923076923
[[74 32]
 [32 70]]
Accuracy: 62.08 %
Standard Deviation: 4.09 %


---------MinMax Scaled---------


Best Accuracy: 0.6971153846153846
N_neighbors:  5
Accuracy for training set -> 0.7702702702702703
Accuracy for test set -> 0.6971153846153846
[[75 31]
 [32 70]]
Accuracy: 64.50 %
Standard Deviation: 5.38 %


['AC', 'NDH', 'HTM', 'Minimum pitch', 'Maximum pitch', 'Number of pulses', 'Number of periods', 'Mean period', 'Fraction of locally unvoiced frames', 'Number of voice breaks', 'Degree of voice breaks', 'Jitter(local, absolute)', 'Shimmer (apq11)', 'Mean Pitch']


Added columns to the dataset are: Jitter(local, absolute) Shimmer (apq11) Mean Pitch


---------Standard Scaled---------


Best Accuracy: 0.6875
N_neighbors:  18
Accuracy for training set -> 0.6978764478764479
Accuracy for test set -> 0.6875
[[74 32]
 [33 69]]
Accuracy: 62.81 %
Standar

Best Accuracy: 0.6971153846153846
N_neighbors:  18
Accuracy for training set -> 0.7036679536679536
Accuracy for test set -> 0.6971153846153846
[[74 32]
 [31 71]]
Accuracy: 62.93 %
Standard Deviation: 5.01 %


---------MinMax Scaled---------


Best Accuracy: 0.6778846153846154
N_neighbors:  9
Accuracy for training set -> 0.7123552123552124
Accuracy for test set -> 0.6778846153846154
[[75 31]
 [36 66]]
Accuracy: 60.52 %
Standard Deviation: 5.00 %


['AC', 'NDH', 'HTM', 'Minimum pitch', 'Maximum pitch', 'Number of pulses', 'Number of periods', 'Mean period', 'Fraction of locally unvoiced frames', 'Number of voice breaks', 'Degree of voice breaks', 'Jitter (rap)', 'Shimmer (apq5)', 'Mean Pitch']


Added columns to the dataset are: Jitter (rap) Shimmer (apq5) Mean Pitch


---------Standard Scaled---------


Best Accuracy: 0.6875
N_neighbors:  25
Accuracy for training set -> 0.6766409266409267
Accuracy for test set -> 0.6875
[[67 39]
 [26 76]]
Accuracy: 62.44 %
Standard Deviation: 5.52 %


-

Accuracy: 63.89 %
Standard Deviation: 4.43 %


---------MinMax Scaled---------


Best Accuracy: 0.6826923076923077
N_neighbors:  11
Accuracy for training set -> 0.7113899613899614
Accuracy for test set -> 0.6826923076923077
[[74 32]
 [34 68]]
Accuracy: 62.45 %
Standard Deviation: 3.76 %


['AC', 'NDH', 'HTM', 'Minimum pitch', 'Maximum pitch', 'Number of pulses', 'Number of periods', 'Mean period', 'Fraction of locally unvoiced frames', 'Number of voice breaks', 'Degree of voice breaks', 'Jitter (ppq5)', 'Shimmer (apq3)', 'Mean Pitch']


Added columns to the dataset are: Jitter (ppq5) Shimmer (apq3) Mean Pitch


---------Standard Scaled---------


Best Accuracy: 0.6971153846153846
N_neighbors:  12
Accuracy for training set -> 0.694980694980695
Accuracy for test set -> 0.6971153846153846
[[80 26]
 [37 65]]
Accuracy: 64.14 %
Standard Deviation: 5.14 %


---------MinMax Scaled---------


Best Accuracy: 0.6778846153846154
N_neighbors:  25
Accuracy for training set -> 0.6708494208494209
Accu

Best Accuracy: 0.7067307692307693
N_neighbors:  12
Accuracy for training set -> 0.7191119691119691
Accuracy for test set -> 0.7067307692307693
[[81 25]
 [36 66]]
Accuracy: 63.89 %
Standard Deviation: 3.60 %


['AC', 'NDH', 'HTM', 'Minimum pitch', 'Maximum pitch', 'Number of pulses', 'Number of periods', 'Mean period', 'Fraction of locally unvoiced frames', 'Number of voice breaks', 'Degree of voice breaks', 'Jitter (ddp)', 'Shimmer (local, db)', 'Mean Pitch']


Added columns to the dataset are: Jitter (ddp) Shimmer (local, db) Mean Pitch


---------Standard Scaled---------


Best Accuracy: 0.7115384615384616
N_neighbors:  18
Accuracy for training set -> 0.7065637065637066
Accuracy for test set -> 0.7115384615384616
[[72 34]
 [26 76]]
Accuracy: 64.00 %
Standard Deviation: 4.56 %


---------MinMax Scaled---------


Best Accuracy: 0.7019230769230769
N_neighbors:  17
Accuracy for training set -> 0.7065637065637066
Accuracy for test set -> 0.7019230769230769
[[74 32]
 [30 72]]
Accuracy: 63.

In [31]:
X1

array([[9.05905e-01, 1.19116e-01, 1.11300e+01, ..., 2.69900e+00,
        1.35510e+01, 1.64781e+02],
       [9.51285e-01, 6.67580e-02, 1.74030e+01, ..., 1.05900e+00,
        6.17500e+00, 1.93289e+02],
       [9.11508e-01, 1.09983e-01, 1.22120e+01, ..., 2.19600e+00,
        1.30400e+01, 1.64768e+02],
       ...,
       [8.83146e-01, 1.61736e-01, 1.10350e+01, ..., 3.42300e+00,
        1.93000e+01, 1.28388e+02],
       [7.65474e-01, 4.23034e-01, 6.58300e+00, ..., 5.35900e+00,
        1.91230e+01, 1.35943e+02],
       [9.25554e-01, 9.74810e-02, 1.34720e+01, ..., 2.04000e+00,
        8.06900e+00, 1.21630e+02]])

In [30]:
min_max_scaler = preprocessing.MinMaxScaler()
minmax_scale_X = min_max_scaler.fit_transform(X1)
minmax_scale_X

array([[0.79927695, 0.13493302, 0.37640227, ..., 0.10535138, 0.15878427,
        0.21236662],
       [0.89828685, 0.07455508, 0.60267648, ..., 0.03662574, 0.06169459,
        0.28582324],
       [0.81150156, 0.12440107, 0.41543123, ..., 0.08427272, 0.15205802,
        0.21233313],
       ...,
       [0.74962146, 0.18408134, 0.37297551, ..., 0.13569124, 0.23445789,
        0.11859271],
       [0.49288516, 0.48540369, 0.21238683, ..., 0.21682102, 0.23212805,
        0.13805969],
       [0.84214706, 0.10998407, 0.46088086, ..., 0.07773541, 0.08662516,
        0.10117936]])

In [15]:
fixed_cols

['AC',
 'NDH',
 'HTM',
 'Minimum pitch',
 'Maximum pitch',
 'Number of pulses',
 'Number of periods',
 'Mean period',
 'Fraction of locally unvoiced frames',
 'Number of voice breaks',
 'Degree of voice breaks']

In [16]:
getKNN(standard_scaled_X, y)

Best Accuracy: 0.7115384615384616
N_neighbors:  9
Accuracy for training set -> 0.7364864864864865
Accuracy for test set -> 0.7115384615384616
[[70 36]
 [24 78]]
Accuracy: 64.62 %
Standard Deviation: 3.53 %


In [17]:
getKNN(minmax_scaled_X, y)

Best Accuracy: 0.7307692307692307
N_neighbors:  10
Accuracy for training set -> 0.7393822393822393
Accuracy for test set -> 0.7307692307692307
[[84 22]
 [34 68]]
Accuracy: 65.33 %
Standard Deviation: 4.22 %


In [32]:
minmax_scaled_X

array([[0.09149866, 0.108709  , 0.1053558 , ..., 0.        , 0.        ,
        0.        ],
       [0.03792471, 0.04051754, 0.03658537, ..., 0.02548833, 0.        ,
        0.        ],
       [0.0726068 , 0.08770943, 0.08423435, ..., 0.12087389, 0.08333333,
        0.00257534],
       ...,
       [0.19187932, 0.28529654, 0.13565502, ..., 0.2268654 , 0.08333333,
        0.06617764],
       [0.25363034, 0.34850561, 0.21674629, ..., 0.18905828, 0.25      ,
        0.28321542],
       [0.1141266 , 0.18440848, 0.07769676, ..., 0.20624334, 0.08333333,
        0.04900386]])

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, Y_train, Y_test = train_test_split(minmax_scaled_X, y, random_state=12, test_size = 0.2)
# Train
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=10)
print(model)
model.fit(X_train, Y_train)

# Evaluate
print("Accuracy for training set -> ", end = "")
Y_hat = [round(yhat) for yhat in model.predict(minmax_scaled_X)]
print(accuracy_score(y, Y_hat)) # Full set accuracy
Y_hat = [round(yhat) for yhat in model.predict(X_test)]
print("Accuracy for test set -> ", end = "")
print(accuracy_score(Y_test, Y_hat)) # Test set accuracy

#Validations
y_pred = model.predict(X_test)

#Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred) # Calulate Confusion matrix for test set.
print(cm)

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = model, X = X_train, y = Y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')
Accuracy for training set -> 0.7393822393822393
Accuracy for test set -> 0.7307692307692307
[[84 22]
 [34 68]]
Accuracy: 65.33 %
Standard Deviation: 4.22 %


In [34]:
standard_scaled_X.shape

(1036, 26)

In [35]:
import joblib
joblib.dump(model, 'finalModel_Deployment.pkl')
print("Model dumped!")

Model dumped!


In [36]:
model_columns = list(data.columns)
model_columns.remove('status')
model_columns

['Jitter(local)',
 'Jitter(local, absolute)',
 'Jitter (rap)',
 'Jitter (ppq5)',
 'Jitter (ddp)',
 'Shimmer (local)',
 'Shimmer (local, db)',
 'Shimmer (apq3)',
 'Shimmer (apq5)',
 'Shimmer (apq11)',
 'Shimmer (dda)',
 'AC',
 'NDH',
 'HTM',
 'Median Pitch',
 'Mean Pitch',
 'Standard deviation',
 'Minimum pitch',
 'Maximum pitch',
 'Number of pulses',
 'Number of periods',
 'Mean period',
 'Standard deviation of period',
 'Fraction of locally unvoiced frames',
 'Number of voice breaks',
 'Degree of voice breaks']

In [37]:
joblib.dump(model_columns, 'model_columns_final.pkl')
print("Models columns dumped!")

Models columns dumped!


In [38]:
data.columns

Index(['Jitter(local)', 'Jitter(local, absolute)', 'Jitter (rap)',
       'Jitter (ppq5)', 'Jitter (ddp)', 'Shimmer (local)',
       'Shimmer (local, db)', 'Shimmer (apq3)', 'Shimmer (apq5)',
       'Shimmer (apq11)', 'Shimmer (dda)', 'AC', 'NDH', 'HTM', 'Median Pitch',
       'Mean Pitch', 'Standard deviation', 'Minimum pitch', 'Maximum pitch',
       'Number of pulses', 'Number of periods', 'Mean period',
       'Standard deviation of period', 'Fraction of locally unvoiced frames',
       'Number of voice breaks', 'Degree of voice breaks', 'status'],
      dtype='object')

In [39]:
minmax_scaled_X.shape

(1036, 26)